# A study on the spatial variability of the uncertainty and fitness-for-purpose of the GrIS ice velocity product.

## Import packages

In [ ]:
import matplotlib.colors
import matplotlib.pyplot as plt
from c3s_eqc_automatic_quality_control import download

plt.style.use("seaborn-v0_8-notebook")

## Define Parameters

In [ ]:
period_start = "2017_2018"
period_stop = "2019_2020"
assert all("_" in period and len(period) == 9 for period in (period_start, period_stop))

## Define request

In [ ]:
y0_start, y1_start = map(int, period_start.split("_"))
y0_stop, y1_stop = map(int, period_stop.split("_"))
collection_id = "satellite-greenland-ice-sheet-velocity"
request = {
    "variable": "all",
    "format": "zip",
    "version": "1.3",
    "period": [
        f"{y0}_{y1}"
        for y0, y1 in zip(range(y0_start, y0_stop + 1), range(y1_start, y1_stop + 1))
    ],
}

## Function to cache

In [ ]:
def mask_high_vel_and_compute_std(ds):
    ds = ds.chunk(y=1351, x=748)

    # Mask out velocities > 1.E3 m/day
    name = "land_ice_surface_velocity_magnitude"
    ds[name] = ds[name].where(ds[name] <= 1_000)

    # Add std
    std = (
        ds["land_ice_surface_easting_stddev"] ** 2
        + ds["land_ice_surface_northing_stddev"] ** 2
    ) ** (1 / 2)
    std.attrs = {
        "long_name": "Velocity standard deviation",
        "units": ds[name].attrs["units"],
    }
    ds["land_ice_surface_stddev"] = std

    # Preserve chunksizes
    ds = ds[
        [
            "land_ice_surface_velocity_magnitude",
            "land_ice_surface_measurement_count",
            "land_ice_surface_stddev",
        ]
    ]
    for da in ds.data_vars.values():
        da.encoding["chunksizes"] = tuple(map(max, da.chunks))
    return ds

## Download and transform

In [ ]:
ds = download.download_and_transform(
    collection_id,
    request,
    chunks={"period": 1},
    transform_func=mask_high_vel_and_compute_std,
)
ds = ds.chunk(y=1351, x=748)

# Rename source dim
periods = []
for source in ds["source"].values.tolist():
    start, stop = source.split("_")[5:7]
    periods.append(f"{start[:4]}-{stop[:4]}")
ds["source"] = periods
ds = ds.rename(source="period")

# Add attributes
for name, da in ds.data_vars.items():
    if name == "land_ice_surface_measurement_count":
        da.attrs["units"] = "pixels"
    if "long_name" not in da.attrs:
        da.attrs["long_name"] = da.attrs.get("description", name.replace("_", " "))

## Plotting function

In [ ]:
def plot_maps_and_print_stats(da, suptitle=None, **kwargs):
    print(f"Mean {da.attrs['long_name']} [{da.attrs['units']}]")
    print(da.mean(("x", "y")).to_pandas())

    kwargs.setdefault("cmap", "rainbow")
    kwargs.setdefault("col", "period" if "period" in da.dims else None)
    if kwargs["col"]:
        kwargs.setdefault("subplot_kws", {})
        kwargs["subplot_kws"].setdefault("aspect", "equal")
    plot_obj = da.plot.imshow(**kwargs)
    if kwargs["col"]:
        for ax in plot_obj.axs.flatten():
            ax.axis("off")
    else:
        plt.axis("equal")
        plt.axis("off")
    if suptitle:
        plt.suptitle(suptitle, y=0, va="top")
    return plot_obj

## Plot velocity

In [ ]:
da = ds["land_ice_surface_velocity_magnitude"]
_ = plot_maps_and_print_stats(
    da,
    norm=matplotlib.colors.LogNorm(),
    vmin=1e-3,
    vmax=20,
    suptitle="Annually averaged horizontal surface velocity magnitude of the Greenland Ice Sheet.",
)

## Plot zoom

In [ ]:
da = ds["land_ice_surface_velocity_magnitude"].isel(
    period=0, x=slice(1600, 3800), y=slice(2000, 3400)
)
_ = plot_maps_and_print_stats(
    da,
    norm=matplotlib.colors.LogNorm(),
    vmin=1e-3,
    vmax=1,
    suptitle=f"Zoom onto the velocity field of the interior of the ice sheet ({da['period'].values}).",
)

## Plot pixel count

In [ ]:
da = ds["land_ice_surface_measurement_count"]
_ = plot_maps_and_print_stats(
    da,
    suptitle="Valid pixel count maps for the ice velocity of the Greenland Ice Sheet.",
)

## Plot velocity standard deviation

In [ ]:
da = ds["land_ice_surface_stddev"]
_ = plot_maps_and_print_stats(
    da,
    norm=matplotlib.colors.LogNorm(),
    vmin=1e-3,
    vmax=1e-1,
    suptitle="Horizontal velocity standard deviation of the Greenland Ice Sheet.",
)

## Velocity standard deviation VS magnitude

In [ ]:
ds_to_plot = ds.isel(period=-1)

names = ["land_ice_surface_stddev", "land_ice_surface_velocity_magnitude"]
ds_to_plot = ds_to_plot[names].stack(xy=("x", "y"))
mask = ds_to_plot[names[-1]].notnull()

fig, ax = plt.subplots()
ax.plot(
    *ds_to_plot.where(mask.compute(), drop=True).data_vars.values(),
    linestyle="none",
    marker="o",
    markersize=5,
)
for axis, name in zip(("x", "y"), names):
    ds = ds_to_plot[name]
    label = f"{da.attrs['long_name']} [{da.attrs['units']}]"
    getattr(ax, f"set_{axis}label")(label)
    getattr(ax, f"set_{axis}scale")("log")
ax.grid()
ax.set_title("Velocity standard deviation vs. velocity magnitude")
_ = fig.suptitle(
    f"Scatter plot of the velocity standard deviation vs. magnitude ({ds_to_plot['period'].values}).",
    y=0,
    va="top",
)